# **Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

## **Edición 2023**


----

# Trabajo práctico entregable - parte 2


En el ejercicio 1 de la parte 1 del entregable seleccionaron las filas y columnas relevantes al problema de predicción de precios de una propiedad. Además de ello, tuvieron que reducir el número de valores posibles para las variables categóricas utilizando información de dominio.

En el ejercicio 2 de la parte 1 del entregable imputaron los valores faltantes de las columnas `Suburb` y las columnas obtenidas a partir del conjunto de datos `airbnb`.

En esta notebook, **se utilizará resultado de dichas operaciones.**


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sklearn
import seaborn as sns


seaborn.set_context('talk')

NameError: name 'seaborn' is not defined

In [2]:
# Acá deberían leer el conjunto de datos que ya tienen.
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Ejercicio 1: Encoding

1. Seleccionar todas las filas y columnas del conjunto de datos obtenido en la parte 1 del entregable, **excepto** `BuildingArea` y `YearBuilt`, que volveremos a imputar más adelante.

2. Aplicar una codificación One-hot encoding a cada fila, tanto para variables numéricas como categóricas. Si lo consideran necesario, pueden volver a reducir el número de categorías únicas.

Algunas opciones:
  1. Utilizar `OneHotEncoder` junto con el parámetro `categories` para las variables categóricas y luego usar `numpy.hstack` para concatenar el resultado con las variables numéricas. 
  2. `DictVectorizer` con algunos pasos de pre-proceso previo.

Recordar también que el atributo `pandas.DataFrame.values` permite acceder a la matriz de numpy subyacente a un DataFrame.


In [25]:
merged_sales_df=pd.read_csv('merged_sales_df.csv')
columns_to_exclude = ['BuildingArea', 'YearBuilt']  # Lista de columnas a excluir
merged_sales_df = merged_sales_df.drop(columns_to_exclude , axis=1)
merged_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Postcode                          6162 non-null   float64
 1   Price                             6162 non-null   float64
 2   CouncilArea                       5621 non-null   object 
 3   Suburb                            6162 non-null   object 
 4   Rooms                             6162 non-null   int64  
 5   Type                              6162 non-null   object 
 6   Bedroom2                          6162 non-null   float64
 7   Bathroom                          6162 non-null   float64
 8   Car                               6162 non-null   float64
 9   Distance                          6162 non-null   float64
 10  Landsize                          6162 non-null   float64
 11  zipcode                           6091 non-null   float64
 12  airbnb

In [106]:
melb_df=merged_sales_df

In [107]:
categorical_cols=['CouncilArea','Suburb','Type']


print(melb_df[categorical_cols].nunique())
num_of_cols_cat=len(categorical_cols)
num_of_cols=melb_df.shape[1]
num_of_new_cols=melb_df[categorical_cols].nunique().sum()
print(num_of_cols_cat,num_of_cols,num_of_new_cols)

# Check for nulls
melb_df[categorical_cols].isna().sum()

melb_df.loc[:,['CouncilArea','Suburb','Type']].dropna()


CouncilArea     28
Suburb         267
Type             3
dtype: int64
3 19 298


,CouncilArea,Suburb,Type
0,Yarra,Abbotsford,h
1,Yarra,Abbotsford,h
2,Yarra,Abbotsford,h
3,Yarra,Abbotsford,h
4,Yarra,Abbotsford,h
...,...,...,...
6126,Maribyrnong,Maidstone,t
6127,Maribyrnong,Maidstone,t
6128,Moreland,Oak Park,t
6129,Darebin,Reservoir,t


In [108]:
# Creamos las variables binarias
dummies = pd.get_dummies(melb_df.loc[:,['CouncilArea','Suburb','Type']], drop_first = False)
dummies.head()
# Añadimos las variables binarias al DataFrame
melb_df = pd.concat([melb_df, dummies], axis = 1)
print('cantidad de columnas total',num_of_new_cols+num_of_cols,'sin quitar las varibles originales')
#elminimanos las variables originales
melb_df = melb_df.drop(columns=['CouncilArea','Suburb','Type'])
print(melb_df.shape[1])
print('cantidad de columnas total',num_of_new_cols+num_of_cols-num_of_cols_cat,'quitando las varibles originales')

cantidad de columnas total 317 sin quitar las varibles originales
314
cantidad de columnas total 314 quitando las varibles originales


In [109]:
melb_df

,Postcode,Price,Rooms,Bedroom2,Bathroom,Car,Distance,Landsize,zipcode,airbnb_price_mean,...,Suburb_Williamstown,Suburb_Williamstown North,Suburb_Windsor,Suburb_Wollert,Suburb_Wyndham Vale,Suburb_Yallambie,Suburb_Yarraville,Type_h,Type_t,Type_u
0,3067.0,1035000.0,2,2.0,1.0,0.0,2.5,156.0,3067.0,130.624031,...,False,False,False,False,False,False,False,True,False,False
1,3067.0,1465000.0,3,3.0,2.0,0.0,2.5,134.0,3067.0,130.624031,...,False,False,False,False,False,False,False,True,False,False
2,3067.0,1600000.0,4,3.0,1.0,2.0,2.5,120.0,3067.0,130.624031,...,False,False,False,False,False,False,False,True,False,False
3,3067.0,1876000.0,3,4.0,2.0,0.0,2.5,245.0,3067.0,130.624031,...,False,False,False,False,False,False,False,True,False,False
4,3067.0,1636000.0,2,2.0,1.0,2.0,2.5,256.0,3067.0,130.624031,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,3147.0,1650000.0,4,4.0,2.0,1.0,10.2,338.0,3147.0,82.758621,...,False,False,False,False,False,False,False,False,True,False
6158,3165.0,1130000.0,4,4.0,2.0,2.0,13.8,270.0,3165.0,113.039216,...,False,False,False,False,False,False,False,False,True,False
6159,3057.0,959000.0,3,3.0,2.0,1.0,4.0,138.0,3057.0,154.432432,...,False,False,False,False,False,False,False,False,True,False
6160,3149.0,1035000.0,3,3.0,2.0,2.0,14.2,236.0,3149.0,119.415584,...,False,False,False,False,False,False,False,False,True,False


In [111]:
melb_df=merged_sales_df
melb_df.loc[:,['CouncilArea','Suburb','Type']].dropna()


,CouncilArea,Suburb,Type
0,Yarra,Abbotsford,h
1,Yarra,Abbotsford,h
2,Yarra,Abbotsford,h
3,Yarra,Abbotsford,h
4,Yarra,Abbotsford,h
...,...,...,...
6126,Maribyrnong,Maidstone,t
6127,Maribyrnong,Maidstone,t
6128,Moreland,Oak Park,t
6129,Darebin,Reservoir,t


In [114]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
encoder.fit(melb_df[categorical_cols])
# We can inspect the categories found by the encoder 
encoder.categories_

[array(['Banyule', 'Bayside', 'Boroondara', 'Brimbank', 'Casey', 'Darebin',
        'Glen Eira', 'Greater Dandenong', 'Hobsons Bay', 'Hume',
        'Kingston', 'Knox', 'Manningham', 'Maribyrnong', 'Maroondah',
        'Melbourne', 'Melton', 'Monash', 'Moonee Valley', 'Moreland',
        'Nillumbik', 'Port Phillip', 'Stonnington', 'Whitehorse',
        'Whittlesea', 'Wyndham', 'Yarra', 'Yarra Ranges', nan],
       dtype=object),
 array(['Abbotsford', 'Aberfeldie', 'Airport West', 'Albanvale',
        'Albert Park', 'Albion', 'Alphington', 'Altona', 'Altona Meadows',
        'Altona North', 'Ardeer', 'Armadale', 'Ascot Vale', 'Ashburton',
        'Ashwood', 'Aspendale', 'Avondale Heights', 'Balaclava', 'Balwyn',
        'Balwyn North', 'Bayswater', 'Bayswater North', 'Beaumaris',
        'Bellfield', 'Bentleigh', 'Bentleigh East', 'Black Rock',
        'Blackburn', 'Blackburn North', 'Blackburn South', 'Bonbeach',
        'Boronia', 'Box Hill', 'Braybrook', 'Briar Hill', 'Brighton',
   

In [117]:
encoded_types = encoder.transform(melb_df.loc[:,['CouncilArea','Suburb','Type']])
encoded_types[:10]

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [119]:
feature_cols = ['Suburb', 'Type', 'Distance', 'Bedroom2', 'Bathroom',
            'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Regionname']
feature_dict = list(melb_df[categorical_cols].T.to_dict().values())
feature_dict[:200]

[{'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abbotsford', 'Type': 'h'},
 {'CouncilArea': 'Yarra', 'Suburb': 'Abb

In [127]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
feature_matrix = vec.fit_transform(feature_dict)

In [128]:
type(feature_matrix)

scipy.sparse._csr.csr_matrix

In [129]:
vec.get_feature_names_out()[:10] 

array(['CouncilArea', 'CouncilArea=Banyule', 'CouncilArea=Bayside',
       'CouncilArea=Boroondara', 'CouncilArea=Brimbank',
       'CouncilArea=Casey', 'CouncilArea=Darebin',
       'CouncilArea=Glen Eira', 'CouncilArea=Greater Dandenong',
       'CouncilArea=Hobsons Bay'], dtype=object)

In [125]:
melb_df

,Postcode,Price,CouncilArea,Suburb,Rooms,Type,Bedroom2,Bathroom,Car,Distance,Landsize,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean,airbnb_review_scores_rating_mean,airbnb_minimum_nights_mean,airbnb_maximum_nights_mean
0,3067.0,1035000.0,Yarra,Abbotsford,2,h,2.0,1.0,0.0,2.5,156.0,3067.0,130.624031,258.0,605.152174,2187.032258,95.288462,4.093023,745.151163
1,3067.0,1465000.0,Yarra,Abbotsford,3,h,3.0,2.0,0.0,2.5,134.0,3067.0,130.624031,258.0,605.152174,2187.032258,95.288462,4.093023,745.151163
2,3067.0,1600000.0,Yarra,Abbotsford,4,h,3.0,1.0,2.0,2.5,120.0,3067.0,130.624031,258.0,605.152174,2187.032258,95.288462,4.093023,745.151163
3,3067.0,1876000.0,Yarra,Abbotsford,3,h,4.0,2.0,0.0,2.5,245.0,3067.0,130.624031,258.0,605.152174,2187.032258,95.288462,4.093023,745.151163
4,3067.0,1636000.0,Yarra,Abbotsford,2,h,2.0,1.0,2.0,2.5,256.0,3067.0,130.624031,258.0,605.152174,2187.032258,95.288462,4.093023,745.151163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,3147.0,1650000.0,NaN,Ashburton,4,t,4.0,2.0,1.0,10.2,338.0,3147.0,82.758621,29.0,520.000000,NaN,96.578947,2.206897,665.517241
6158,3165.0,1130000.0,NaN,Bentleigh East,4,t,4.0,2.0,2.0,13.8,270.0,3165.0,113.039216,51.0,563.200000,2032.800000,93.962963,5.509804,814.823529
6159,3057.0,959000.0,NaN,Brunswick East,3,t,3.0,2.0,1.0,4.0,138.0,3057.0,154.432432,185.0,595.842105,2789.666667,94.194030,3.627027,708.962162
6160,3149.0,1035000.0,NaN,Mount Waverley,3,t,3.0,2.0,2.0,14.2,236.0,3149.0,119.415584,77.0,450.285714,1588.500000,94.576923,5.363636,807.246753


## Ejercicio 2: Imputación por KNN

En el teórico se presentó el método `IterativeImputer` para imputar valores faltantes en variables numéricas. Sin embargo, los ejemplos presentados sólo utilizaban algunas variables numéricas presentes en el conjunto de datos. En este ejercicio, utilizaremos la matriz de datos codificada para imputar datos faltantes de manera más precisa.

1. Agregue a la matriz obtenida en el punto anterior las columnas `YearBuilt` y `BuildingArea`.
2. Aplique una instancia de `IterativeImputer` con un estimador `KNeighborsRegressor` para imputar los valores de las variables. ¿Es necesario estandarizar o escalar los datos previamente?
3. Realice un gráfico mostrando la distribución de cada variable antes de ser imputada, y con ambos métodos de imputación.

In [ ]:
!pip install scikit-learn

In [ ]:
import sklearn as sklearn

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import IterativeImputer

melb_data_mice = melb_df.copy(deep=True)

mice_imputer = IterativeImputer(random_state=0, estimator=KNeighborsRegressor())
melb_data_mice[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(
    melb_data_mice[['YearBuilt', 'BuildingArea']])

Ejemplo de gráfico comparando las distribuciones de datos obtenidas con cada método de imputación.

In [ ]:
mice_year_built = melb_data_mice.YearBuilt.to_frame()
mice_year_built['Imputation'] = 'KNN over YearBuilt and BuildingArea'
melb_year_build = melb_df.YearBuilt.dropna().to_frame()
melb_year_build['Imputation'] = 'Original'
data = pandas.concat([mice_year_built, melb_year_build])
fig = plt.figure(figsize=(8, 5))
g = seaborn.kdeplot(data=data, x='YearBuilt', hue='Imputation')

## Ejercicio 3: Reducción de dimensionalidad.

Utilizando la matriz obtenida en el ejercicio anterior:
1. Aplique `PCA` para obtener $n$ componentes principales de la matriz, donde `n = min(20, X.shape[0])`. ¿Es necesario estandarizar o escalar los datos?
2. Grafique la varianza capturada por los primeros $n$ componentes principales, para cada $n$.
3. En base al gráfico, seleccione las primeras $m$ columnas de la matriz transformada para agregar como nuevas características al conjunto de datos.

## Ejercicio 4: Composición del resultado

Transformar nuevamente el conjunto de datos procesado en un `pandas.DataFrame` y guardarlo en un archivo.

Para eso, será necesario recordar el nombre original de cada columna de la matriz, en el orden correcto. Tener en cuenta:
1. El método `OneHotEncoder.get_feature_names` o el atributo `OneHotEncoder.categories_` permiten obtener una lista con los valores de la categoría que le corresponde a cada índice de la matriz.
2. Ninguno de los métodos aplicados intercambia de lugar las columnas o las filas de la matriz.

In [ ]:
## Small example
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

## If we process our data with the following steps:
categorical_cols = ['Type', 'Regionname']
numerical_cols = ['Rooms', 'Distance']
new_columns = []

# Step 1: encode categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = encoder.fit_transform(melb_df[categorical_cols])
for col, col_values in zip(categorical_cols, encoder.categories_):
  for col_value in col_values:
    new_columns.append('{}={}'.format(col, col_value))
print("Matrix has shape {}, with columns: {}".format(X_cat.shape, new_columns))

# Step 2: Append the numerical columns
X = numpy.hstack([X_cat, melb_df[numerical_cols].values])
new_columns.extend(numerical_cols)
print("Matrix has shape {}, with columns: {}".format(X_cat.shape, new_columns))

# Step 3: Append some new features, like PCA
pca = PCA(n_components=2)
pca_dummy_features = pca.fit_transform(X)
X_pca = numpy.hstack([X, pca_dummy_features])
new_columns.extend(['pca1', 'pca2'])

## Re-build dataframe
processed_melb_df = pandas.DataFrame(data=X_pca, columns=new_columns)
processed_melb_df.head()

## Ejercicio 5: Documentación

En un documento `.pdf` o `.md` realizar un reporte de las operaciones que realizaron para obtener el conjunto de datos final. Se debe incluir:
  1. Criterios de exclusión (o inclusión) de filas
  2. Interpretación de las columnas presentes
  2. Todas las transofrmaciones realizadas

Este documento es de uso técnico exclusivamente, y su objetivo es permitir que otres desarrolladores puedan reproducir los mismos pasos y obtener el mismo resultado. Debe ser detallado pero consiso. Por ejemplo:

```
  ## Criterios de exclusión de ejemplos
  1. Se eliminan ejemplos donde el año de construcción es previo a 1900

  ## Características seleccionadas
  ### Características categóricas
  1. Type: tipo de propiedad. 3 valores posibles
  2. ...
  Todas las características categóricas fueron codificadas con un
  método OneHotEncoding utilizando como máximo sus 30 valores más 
  frecuentes.
  
  ### Características numéricas
  1. Rooms: Cantidad de habitaciones
  2. Distance: Distancia al centro de la ciudad.
  3. airbnb_mean_price: Se agrega el precio promedio diario de 
     publicaciones de la plataforma AirBnB en el mismo código 
     postal. [Link al repositorio con datos externos].

  ### Transformaciones:
  1. Todas las características numéricas fueron estandarizadas.
  2. La columna `Suburb` fue imputada utilizando el método ...
  3. Las columnas `YearBuilt` y ... fueron imputadas utilizando el 
     algoritmo ...
  4. ...

  ### Datos aumentados
  1. Se agregan las 5 primeras columnas obtenidas a través del
     método de PCA, aplicado sobre el conjunto de datos
     totalmente procesado.
```
